In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

pd.options.display.max_columns, pd.options.display.max_rows = 500, 200
pd.options.display.float_format = '{:.3f}'.format
import warnings
warnings.filterwarnings('ignore')

from soccer.outcomes_features import TrainCreator
from soccer.preprocessing import DataPreprocessor
from soccer.outcomes_catboost import OutcomesCatBoost
from soccer.outcomes_lgbm import OutcomesLGBM

In [3]:
matches = DataPreprocessor(is_boosting_train=False).preprocessing()

[LightGBM] [Warning] lambda_l1 is set=2.05114, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.05114
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=7.23524, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.23524
[LightGBM] [Warning] bagging_fraction is set=0.772, subsample=1.0 will be ignored. Current value: bagging_fraction=0.772
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.35, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.35


In [4]:
train, validation, test = TrainCreator().train_validation_test(matches)

In [5]:
train['season'].max(), validation['season'].max(), test['season'].min(), test['season'].max()

(2020, 2020, 2021, 2022)

In [6]:
OutcomesCatBoost().cross_val_score()

Training on fold [0/3]

bestTest = 0.8794734727
bestIteration = 998

Training on fold [1/3]

bestTest = 0.8785374681
bestIteration = 947

Training on fold [2/3]

bestTest = 0.8775554527
bestIteration = 999



0.8785274144332368

In [7]:
OutcomesCatBoost().save_model()

In [8]:
validation = OutcomesCatBoost().validation()

MSE:  1.415918917498779
MAE:  0.9263701895222806


In [9]:
# experiments = OutcomesCatBoost().optuna_optimization(100)

In [10]:
predictions = OutcomesCatBoost().predict()
predictions.head()

,match_id,home_goals,away_goals,home_win,draw,away_win
0,186911,1.731,0.832,0.587,0.235,0.178
1,186910,1.468,0.991,0.482,0.263,0.255
2,186909,1.210,0.992,0.410,0.289,0.301
3,186908,1.243,1.343,0.345,0.264,0.391
4,186907,1.737,0.873,0.578,0.235,0.187


In [11]:
matches = DataPreprocessor(is_boosting_train=False).preprocessing()
train, validation, test = TrainCreator().train_validation_test(matches)

In [12]:
matches.shape

(363415, 13)

In [13]:
OutcomesLGBM().cross_val_score() # 8767173807530748

0.8765695168140738

In [14]:
OutcomesLGBM().save_model()

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3585]	valid_0's poisson: 0.880691


In [15]:
validation = OutcomesLGBM().validation()
validation.head()

MSE:  1.4137228438904672
MAE:  0.9260970657765656


,is_home,is_pandemic,avg_scoring_5,avg_scoring_10,avg_scoring_20,avg_scoring_30,avg_scoring_5_against,avg_scoring_10_against,avg_scoring_20_against,avg_scoring_30_against,tournament_type,tournament,league,opp_league,location_mean_score_5,location_mean_score_10,location_mean_score_20,location_mean_score_30,location_median_score_5,location_median_score_10,location_median_score_20,location_median_score_30,location_mean_score_5_against,location_mean_score_10_against,location_mean_score_20_against,location_median_score_10_against,location_max_score_10,score,prediction
0,True,True,1.600,2.500,1.850,1.567,1.800,1.600,1.400,1.200,1,Colombia. First,Colombia. First,Colombia. First,3.000,2.000,1.650,1.600,2.000,1.000,1.000,1.000,0.600,0.500,1.000,0.000,6.000,1,1.783
1,True,True,1.000,1.400,1.400,1.367,1.200,1.700,1.400,1.400,1,Colombia. First,Colombia. First,Colombia. First,1.600,1.200,1.300,1.300,1.000,1.000,1.000,1.000,1.000,0.900,0.850,1.000,3.000,1,1.569
2,True,True,0.800,1.100,1.050,1.167,1.000,0.800,0.800,1.100,1,Colombia. First,Colombia. First,Colombia. First,1.400,1.300,1.650,1.467,1.000,1.000,1.500,1.000,0.800,1.100,0.900,1.000,2.000,2,1.314
3,True,True,1.400,1.400,1.650,1.567,1.000,0.800,1.000,0.900,1,Colombia. First,Colombia. First,Colombia. First,1.200,1.400,1.400,1.633,1.000,1.000,1.000,1.000,1.600,1.700,1.350,2.000,3.000,2,1.287
4,True,True,1.400,1.600,1.650,1.567,1.600,1.300,1.400,1.600,1,Colombia. First,Colombia. First,Colombia. First,1.600,1.800,1.600,1.567,2.000,2.000,2.000,2.000,1.200,1.000,0.900,1.000,3.000,1,1.763


In [18]:
# experiments = OutcomesLGBM().optuna_optimization(200)

In [ ]:
# experiments.head()

In [16]:
predictions = OutcomesLGBM().predict()
predictions.head()

,match_id,home_goals,away_goals,home_win,draw,away_win
0,186911,1.783,0.832,0.599,0.229,0.172
1,186910,1.569,1.001,0.506,0.253,0.241
2,186909,1.314,1.024,0.432,0.277,0.291
3,186908,1.287,1.317,0.361,0.264,0.375
4,186907,1.763,0.876,0.583,0.232,0.184


In [17]:
from soccer.compare_models import compare_models
compare_models()

,model,loss
0,Catboost,0.588
1,LightGBM,0.588
2,Modified Glicko-2,0.583
3,Original Glicko-2,0.594
